In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

In [2]:
filepath = 'dataset/energy_dataset.csv'
df = pd.read_csv(filepath)
print(df.shape)
df.head()

(35064, 29)


,time,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,...,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2015-01-01 00:00:00+01:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,...,196.0,0.0,6378.0,17.0,NaN,6436.0,26118.0,25385.0,50.10,65.41
1,2015-01-01 01:00:00+01:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,...,195.0,0.0,5890.0,16.0,NaN,5856.0,24934.0,24382.0,48.10,64.92
2,2015-01-01 02:00:00+01:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,...,196.0,0.0,5461.0,8.0,NaN,5454.0,23515.0,22734.0,47.33,64.48
3,2015-01-01 03:00:00+01:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,...,191.0,0.0,5238.0,2.0,NaN,5151.0,22642.0,21286.0,42.27,59.32
4,2015-01-01 04:00:00+01:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,...,189.0,0.0,4935.0,9.0,NaN,4861.0,21785.0,20264.0,38.41,56.04


In [3]:
df.tail()

,time,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,...,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
35059,2018-12-31 19:00:00+01:00,297.0,0.0,0.0,7634.0,2628.0,178.0,0.0,0.0,0.0,...,277.0,0.0,3113.0,96.0,NaN,3253.0,30619.0,30653.0,68.85,77.02
35060,2018-12-31 20:00:00+01:00,296.0,0.0,0.0,7241.0,2566.0,174.0,0.0,0.0,0.0,...,280.0,0.0,3288.0,51.0,NaN,3353.0,29932.0,29735.0,68.40,76.16
35061,2018-12-31 21:00:00+01:00,292.0,0.0,0.0,7025.0,2422.0,168.0,0.0,0.0,0.0,...,286.0,0.0,3503.0,36.0,NaN,3404.0,27903.0,28071.0,66.88,74.30
35062,2018-12-31 22:00:00+01:00,293.0,0.0,0.0,6562.0,2293.0,163.0,0.0,0.0,0.0,...,287.0,0.0,3586.0,29.0,NaN,3273.0,25450.0,25801.0,63.93,69.89
35063,2018-12-31 23:00:00+01:00,290.0,0.0,0.0,6926.0,2166.0,163.0,0.0,0.0,0.0,...,287.0,0.0,3651.0,26.0,NaN,3117.0,24424.0,24455.0,64.27,69.88


In [4]:
# Check all nan values

print('Energy Data NaN values: \n', df.isna().sum())

Energy Data NaN values: 
 time                                               0
generation biomass                                19
generation fossil brown coal/lignite              18
generation fossil coal-derived gas                18
generation fossil gas                             18
generation fossil hard coal                       18
generation fossil oil                             19
generation fossil oil shale                       18
generation fossil peat                            18
generation geothermal                             18
generation hydro pumped storage aggregated     35064
generation hydro pumped storage consumption       19
generation hydro run-of-river and poundage        19
generation hydro water reservoir                  18
generation marine                                 19
generation nuclear                                17
generation other                                  18
generation other renewable                        18
generation solar    

In [5]:
# Check the features that have the same number of NaN values as the lenght of the dataframe

def CheckNull(data_frame):
    for i in data_frame.columns.values:
        if data_frame[i].isna().sum() == len(data_frame):
            print('This column is empty: ', i)
            
CheckNull(df)

This column is empty:  generation hydro pumped storage aggregated
This column is empty:  forecast wind offshore eday ahead


In [6]:
#Dropping these columns as they contain no values
df = df.drop(['generation fossil coal-derived gas','generation fossil oil shale', 
                            'generation fossil peat', 'generation geothermal','generation marine', 
                            'generation wind offshore','generation hydro pumped storage aggregated', 'forecast wind offshore eday ahead'], 
                            axis=1)

In [7]:
# Substituting NaN values in energy dataset with linear interpolation
df.interpolate(method='linear', inplace=True, axis=0)

# Checking for duplicated values

duplicatedEnergy_values = df.duplicated().sum()
print('There is {} duplicated values.'.format(duplicatedEnergy_values))

There is 0 duplicated values.


In [8]:
df.shape

(35064, 21)

In [11]:
df['time']=pd.to_datetime(df.loc[:,'time'],utc=True)
# df['time']=pd.to_datetime(df.loc[:,'time']).to_pydatetime()

In [12]:
df

,time,generation biomass,generation fossil brown coal/lignite,generation fossil gas,generation fossil hard coal,generation fossil oil,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation nuclear,...,generation other renewable,generation solar,generation waste,generation wind onshore,forecast solar day ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2014-12-31 23:00:00+00:00,447.0,329.0,4844.0,4821.0,162.0,863.0,1051.0,1899.0,7096.0,...,73.0,49.0,196.0,6378.0,17.0,6436.0,26118.0,25385.0,50.10,65.41
1,2015-01-01 00:00:00+00:00,449.0,328.0,5196.0,4755.0,158.0,920.0,1009.0,1658.0,7096.0,...,71.0,50.0,195.0,5890.0,16.0,5856.0,24934.0,24382.0,48.10,64.92
2,2015-01-01 01:00:00+00:00,448.0,323.0,4857.0,4581.0,157.0,1164.0,973.0,1371.0,7099.0,...,73.0,50.0,196.0,5461.0,8.0,5454.0,23515.0,22734.0,47.33,64.48
3,2015-01-01 02:00:00+00:00,438.0,254.0,4314.0,4131.0,160.0,1503.0,949.0,779.0,7098.0,...,75.0,50.0,191.0,5238.0,2.0,5151.0,22642.0,21286.0,42.27,59.32
4,2015-01-01 03:00:00+00:00,428.0,187.0,4130.0,3840.0,156.0,1826.0,953.0,720.0,7097.0,...,74.0,42.0,189.0,4935.0,9.0,4861.0,21785.0,20264.0,38.41,56.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2018-12-31 18:00:00+00:00,297.0,0.0,7634.0,2628.0,178.0,1.0,1135.0,4836.0,6073.0,...,95.0,85.0,277.0,3113.0,96.0,3253.0,30619.0,30653.0,68.85,77.02
35060,2018-12-31 19:00:00+00:00,296.0,0.0,7241.0,2566.0,174.0,1.0,1172.0,3931.0,6074.0,...,95.0,33.0,280.0,3288.0,51.0,3353.0,29932.0,29735.0,68.40,76.16
35061,2018-12-31 20:00:00+00:00,292.0,0.0,7025.0,2422.0,168.0,50.0,1148.0,2831.0,6076.0,...,94.0,31.0,286.0,3503.0,36.0,3404.0,27903.0,28071.0,66.88,74.30
35062,2018-12-31 21:00:00+00:00,293.0,0.0,6562.0,2293.0,163.0,108.0,1128.0,2068.0,6075.0,...,93.0,31.0,287.0,3586.0,29.0,3273.0,25450.0,25801.0,63.93,69.89


In [13]:
df.set_index('time',inplace=True)

In [38]:
print(type(df.index))

<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


In [14]:
print(df.isna().sum())

generation biomass                             0
generation fossil brown coal/lignite           0
generation fossil gas                          0
generation fossil hard coal                    0
generation fossil oil                          0
generation hydro pumped storage consumption    0
generation hydro run-of-river and poundage     0
generation hydro water reservoir               0
generation nuclear                             0
generation other                               0
generation other renewable                     0
generation solar                               0
generation waste                               0
generation wind onshore                        0
forecast solar day ahead                       0
forecast wind onshore day ahead                0
total load forecast                            0
total load actual                              0
price day ahead                                0
price actual                                   0
dtype: int64


In [15]:
df.rename(columns = {'generation biomass':'gen_biomass',
                    'generation fossil brown coal/lignite':'gen_fossil_coal', 
                    'generation fossil gas':'gen_fossil_gas',
                    'generation fossil hard coal':'gen_fossil_hard_coal',
                    'generation fossil oil':'gen_fossil_oil',
                    'generation hydro pumped storage consumption':'gen_hydro',
                    'generation hydro run-of-river and poundage':'gen_hydro_ror',
                    'generation hydro water reservoir':'gen_hydro_water',
                    'generation nuclear':'gen_nuclear',
                    'generation other':'gen_other',
                    'generation other renewable':'gen_other_renewable',
                    'generation solar':'gen_solar',
                    'generation waste':'gen_waste',
                    'generation wind onshore':'gen_waste_onshore',
                    'forecast solar day ahead':'forecast_solar_ahead',
                    'forecast wind onshore day ahead ':'forecast_wind_onshore',
                    'total load forecast':'total_load_forecast',
                    'total load actual':'total_load_actual',
                    'price day ahead':'price_day_ahead',
                    'price actual':'price_actual',}, 
          inplace = True)

In [19]:
df

,gen_biomass,gen_fossil_coal,gen_fossil_gas,gen_fossil_hard_coal,gen_fossil_oil,gen_hydro,gen_hydro_ror,gen_hydro_water,gen_nuclear,gen_other,gen_other_renewable,gen_solar,gen_waste,gen_waste_onshore,forecast_solar_ahead,forecast wind onshore day ahead,total_load_forecast,total_load_actual,price_day_ahead,price_actual
time,,,,,,,,,,,,,,,,,,,,
2014-12-31 23:00:00+00:00,447.0,329.0,4844.0,4821.0,162.0,863.0,1051.0,1899.0,7096.0,43.0,73.0,49.0,196.0,6378.0,17.0,6436.0,26118.0,25385.0,50.10,65.41
2015-01-01 00:00:00+00:00,449.0,328.0,5196.0,4755.0,158.0,920.0,1009.0,1658.0,7096.0,43.0,71.0,50.0,195.0,5890.0,16.0,5856.0,24934.0,24382.0,48.10,64.92
2015-01-01 01:00:00+00:00,448.0,323.0,4857.0,4581.0,157.0,1164.0,973.0,1371.0,7099.0,43.0,73.0,50.0,196.0,5461.0,8.0,5454.0,23515.0,22734.0,47.33,64.48
2015-01-01 02:00:00+00:00,438.0,254.0,4314.0,4131.0,160.0,1503.0,949.0,779.0,7098.0,43.0,75.0,50.0,191.0,5238.0,2.0,5151.0,22642.0,21286.0,42.27,59.32
2015-01-01 03:00:00+00:00,428.0,187.0,4130.0,3840.0,156.0,1826.0,953.0,720.0,7097.0,43.0,74.0,42.0,189.0,4935.0,9.0,4861.0,21785.0,20264.0,38.41,56.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 18:00:00+00:00,297.0,0.0,7634.0,2628.0,178.0,1.0,1135.0,4836.0,6073.0,63.0,95.0,85.0,277.0,3113.0,96.0,3253.0,30619.0,30653.0,68.85,77.02
2018-12-31 19:00:00+00:00,296.0,0.0,7241.0,2566.0,174.0,1.0,1172.0,3931.0,6074.0,62.0,95.0,33.0,280.0,3288.0,51.0,3353.0,29932.0,29735.0,68.40,76.16
2018-12-31 20:00:00+00:00,292.0,0.0,7025.0,2422.0,168.0,50.0,1148.0,2831.0,6076.0,61.0,94.0,31.0,286.0,3503.0,36.0,3404.0,27903.0,28071.0,66.88,74.30


In [20]:
# cor = df.corr()
# #Correlation with output variable
# cor_target = abs(cor["price actual"])
# #Selecting highly correlated features
# relevant_features = cor_target[cor_target>0.90]
# relevant_features

In [21]:
nobs = 5000
df_train, df_test = df[0:-nobs], df[-nobs:]

# Check size
print(df_train.shape)
print(df_test.shape)  

(30064, 20)
(5000, 20)


In [22]:
# test =df_test[['price actual']]

In [23]:
def adfuller_test(series, signif=0.05, name='', verbose=False):
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")    

In [24]:
for name, column in df_train.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "gen_biomass" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -7.0844
 No. Lags Chosen       = 49
 Critical value 1%     = -3.431
 Critical value 5%     = -2.862
 Critical value 10%    = -2.567
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "gen_fossil_coal" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -13.8117
 No. Lags Chosen       = 50
 Critical value 1%     = -3.431
 Critical value 5%     = -2.862
 Critical value 10%    = -2.567
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "gen_fossil_gas" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.

In [25]:
model = VAR(df)
for i in [1,2,3,4,5]:
    result = model.fit(i)
    print('Lag Order =', i)
    print('AIC : ', result.aic)
    print('BIC : ', result.bic)
    print('FPE : ', result.fpe)
    print('HQIC: ', result.hqic, '\n')

Lag Order = 1
AIC :  165.00199511974046
BIC :  165.10339142705624
FPE :  4.565159805099297e+71
HQIC:  165.03428963676453 



/Users/arpit_singh/Anaconda/anaconda3/envs/ml-poc/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)


Lag Order = 2
AIC :  161.4156967740629
BIC :  161.61366597221385
FPE :  1.2645802966194183e+70
HQIC:  161.478749644768 

Lag Order = 3
AIC :  161.08061182196255
BIC :  161.37515876900724
FPE :  9.045256490398651e+69
HQIC:  161.17442467653427 

Lag Order = 4
AIC :  160.9193182969118
BIC :  161.31044785129654
FPE :  7.697895999821107e+69
HQIC:  161.04389276566937 

Lag Order = 5
AIC :  160.8587399185624
BIC :  161.3464569391215
FPE :  7.24541475183733e+69
HQIC:  161.01407763195877 



In [26]:
x = model.select_order(maxlags=5)
x.summary()

,AIC,BIC,FPE,HQIC
0,214.2,214.2,1.028e+93,214.2
1,165.0,165.1,4.571e+71,165.0
2,161.4,161.6,1.266e+70,161.5
3,161.1,161.4,9.050e+69,161.2
4,160.9,161.3*,7.701e+69,161.0
5,160.9*,161.3,7.245e+69*,161.0*


In [27]:
model_fitted = model.fit()
model_fitted.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Mon, 10, Aug, 2020
Time:                     17:26:15
--------------------------------------------------------------------
No. of Equations:         20.0000    BIC:                    165.103
Nobs:                     35063.0    HQIC:                   165.034
Log likelihood:      -3.88736e+06    FPE:                4.56516e+71
AIC:                      165.002    Det(Omega_mle):     4.51082e+71
--------------------------------------------------------------------
Results for equation gen_biomass
                                        coefficient       std. error           t-stat            prob
-----------------------------------------------------------------------------------------------------
const                                     28.644179         1.249695           22.921           0.000
L1.gen_biomass                             0.963746         0.001433    

In [28]:
# Get the lag order
lag_order = model_fitted.k_ar
print(lag_order)

# Input data for forecasting
forecast_input = df.values
print(forecast_input)
print(len(forecast_input))

1
[[  447.     329.    4844.   ... 25385.      50.1     65.41]
 [  449.     328.    5196.   ... 24382.      48.1     64.92]
 [  448.     323.    4857.   ... 22734.      47.33    64.48]
 ...
 [  292.       0.    7025.   ... 28071.      66.88    74.3 ]
 [  293.       0.    6562.   ... 25801.      63.93    69.89]
 [  290.       0.    6926.   ... 24455.      64.27    69.88]]
35064


In [29]:
# Forecasting 5000 values of test data
fc = model_fitted.forecast(y=forecast_input, steps=5000)
df_forecast = pd.DataFrame(fc, index=df.index[-nobs:], columns=df.columns + '_pred')
df_forecast.tail()

,gen_biomass_pred,gen_fossil_coal_pred,gen_fossil_gas_pred,gen_fossil_hard_coal_pred,gen_fossil_oil_pred,gen_hydro_pred,gen_hydro_ror_pred,gen_hydro_water_pred,gen_nuclear_pred,gen_other_pred,gen_other_renewable_pred,gen_solar_pred,gen_waste_pred,gen_waste_onshore_pred,forecast_solar_ahead_pred,forecast wind onshore day ahead_pred,total_load_forecast_pred,total_load_actual_pred,price_day_ahead_pred,price_actual_pred
time,,,,,,,,,,,,,,,,,,,,
2018-12-31 18:00:00+00:00,382.881538,448.068156,5629.404605,4252.422833,298.162309,474.350347,972.008996,2601.832018,6261.098186,60.136448,85.756346,1433.905953,269.836444,5460.37854,1440.212969,5466.506914,28710.95117,28697.45294,49.915589,57.901342
2018-12-31 19:00:00+00:00,382.881538,448.068156,5629.404605,4252.422833,298.162309,474.350347,972.008996,2601.832018,6261.098186,60.136448,85.756346,1433.905953,269.836444,5460.37854,1440.212969,5466.506914,28710.95117,28697.45294,49.915589,57.901342
2018-12-31 20:00:00+00:00,382.881538,448.068156,5629.404605,4252.422833,298.162309,474.350347,972.008996,2601.832018,6261.098186,60.136448,85.756346,1433.905953,269.836444,5460.37854,1440.212969,5466.506914,28710.95117,28697.45294,49.915589,57.901342
2018-12-31 21:00:00+00:00,382.881538,448.068156,5629.404605,4252.422833,298.162309,474.350347,972.008996,2601.832018,6261.098186,60.136448,85.756346,1433.905953,269.836444,5460.37854,1440.212969,5466.506914,28710.95117,28697.45294,49.915589,57.901342
2018-12-31 22:00:00+00:00,382.881538,448.068156,5629.404605,4252.422833,298.162309,474.350347,972.008996,2601.832018,6261.098186,60.136448,85.756346,1433.905953,269.836444,5460.37854,1440.212969,5466.506914,28710.95117,28697.45294,49.915589,57.901342


In [30]:
from pandas.tseries.offsets import DateOffset
pred_df=[df.index[-1]+ DateOffset(hours=x)for x in range(1,17523)]

In [31]:
pred_df=pd.DataFrame(index=pred_df[1:],columns=df.columns)

In [32]:
pred_df.head()

,gen_biomass,gen_fossil_coal,gen_fossil_gas,gen_fossil_hard_coal,gen_fossil_oil,gen_hydro,gen_hydro_ror,gen_hydro_water,gen_nuclear,gen_other,gen_other_renewable,gen_solar,gen_waste,gen_waste_onshore,forecast_solar_ahead,forecast wind onshore day ahead,total_load_forecast,total_load_actual,price_day_ahead,price_actual
2019-01-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01 01:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01 02:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01 03:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01 04:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
pred_df.shape

(17521, 20)

In [34]:
pred_df.tail()

,gen_biomass,gen_fossil_coal,gen_fossil_gas,gen_fossil_hard_coal,gen_fossil_oil,gen_hydro,gen_hydro_ror,gen_hydro_water,gen_nuclear,gen_other,gen_other_renewable,gen_solar,gen_waste,gen_waste_onshore,forecast_solar_ahead,forecast wind onshore day ahead,total_load_forecast,total_load_actual,price_day_ahead,price_actual
2020-12-30 20:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-30 21:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-30 22:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-30 23:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
final_df=pd.concat([df,pred_df])

In [36]:
final_df

,gen_biomass,gen_fossil_coal,gen_fossil_gas,gen_fossil_hard_coal,gen_fossil_oil,gen_hydro,gen_hydro_ror,gen_hydro_water,gen_nuclear,gen_other,gen_other_renewable,gen_solar,gen_waste,gen_waste_onshore,forecast_solar_ahead,forecast wind onshore day ahead,total_load_forecast,total_load_actual,price_day_ahead,price_actual
2014-12-31 23:00:00+00:00,447.0,329.0,4844.0,4821.0,162.0,863.0,1051.0,1899.0,7096.0,43.0,73.0,49.0,196.0,6378.0,17.0,6436.0,26118.0,25385.0,50.10,65.41
2015-01-01 00:00:00+00:00,449.0,328.0,5196.0,4755.0,158.0,920.0,1009.0,1658.0,7096.0,43.0,71.0,50.0,195.0,5890.0,16.0,5856.0,24934.0,24382.0,48.10,64.92
2015-01-01 01:00:00+00:00,448.0,323.0,4857.0,4581.0,157.0,1164.0,973.0,1371.0,7099.0,43.0,73.0,50.0,196.0,5461.0,8.0,5454.0,23515.0,22734.0,47.33,64.48
2015-01-01 02:00:00+00:00,438.0,254.0,4314.0,4131.0,160.0,1503.0,949.0,779.0,7098.0,43.0,75.0,50.0,191.0,5238.0,2.0,5151.0,22642.0,21286.0,42.27,59.32
2015-01-01 03:00:00+00:00,428.0,187.0,4130.0,3840.0,156.0,1826.0,953.0,720.0,7097.0,43.0,74.0,42.0,189.0,4935.0,9.0,4861.0,21785.0,20264.0,38.41,56.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-30 20:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-30 21:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-30 22:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-30 23:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
start = '2019-01-01 00:00:00+00:00'
end = '2020-12-31 00:00:00+00:00'
t1 = pd.to_datetime(start, utc=True)
print(type(t1))
t2 = pd.to_datetime(end, utc=True)
print(type(t2))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [ ]:
# future_df['forecast'] = results.predict(start = f1, end = f2, dynamic= True)  
# future_df[['price actual', 'forecast']].plot(figsize=(25, 8))

# pred = model_fit.forecast(y=forecast_input, steps=8761)
# pred = (pd.DataFrame(pred, index=pred_df.index, columns=pred_df.columns + '_pred'))
# print(pred)

In [40]:
final = model_fitted.forecast(y=forecast_input, steps=17521)
final = (pd.DataFrame(final,  index=pred_df.index, columns=pred_df.columns))
print(final)

                           gen_biomass  gen_fossil_coal  gen_fossil_gas  \
2019-01-01 00:00:00+00:00   291.530517        20.205801     6954.336013   
2019-01-01 01:00:00+00:00   293.147677        39.775659     6978.980286   
2019-01-01 02:00:00+00:00   294.781897        58.567672     6999.683691   
2019-01-01 03:00:00+00:00   296.386236        76.497144     7016.364713   
2019-01-01 04:00:00+00:00   297.930061        93.520467     7029.058552   
...                                ...              ...             ...   
2020-12-30 20:00:00+00:00   382.881538       448.068156     5629.404605   
2020-12-30 21:00:00+00:00   382.881538       448.068156     5629.404605   
2020-12-30 22:00:00+00:00   382.881538       448.068156     5629.404605   
2020-12-30 23:00:00+00:00   382.881538       448.068156     5629.404605   
2020-12-31 00:00:00+00:00   382.881538       448.068156     5629.404605   

                           gen_fossil_hard_coal  gen_fossil_oil   gen_hydro  \
2019-01-01 00:00:00+

In [41]:
final[:10]

,gen_biomass,gen_fossil_coal,gen_fossil_gas,gen_fossil_hard_coal,gen_fossil_oil,gen_hydro,gen_hydro_ror,gen_hydro_water,gen_nuclear,gen_other,gen_other_renewable,gen_solar,gen_waste,gen_waste_onshore,forecast_solar_ahead,forecast wind onshore day ahead,total_load_forecast,total_load_actual,price_day_ahead,price_actual
2019-01-01 00:00:00+00:00,291.530517,20.205801,6954.336013,2275.536934,171.598416,126.682810,1075.346228,1852.964707,6087.802113,60.915212,92.631661,238.058976,287.661049,3613.436721,249.195858,3166.096689,24905.546320,24935.201036,64.117776,69.786160
2019-01-01 01:00:00+00:00,293.147677,39.775659,6978.980286,2382.385073,179.779698,139.925424,1081.594251,2000.239995,6101.128967,60.824391,93.268655,440.935176,288.363855,3590.562116,463.108246,3212.392668,25366.054435,25396.338887,63.948818,69.694094
2019-01-01 02:00:00+00:00,294.781897,58.567672,6999.683691,2485.185971,187.510425,149.047076,1087.566273,2129.294139,6114.343009,60.721666,93.893662,635.674931,289.073655,3581.169058,665.081553,3258.677603,25802.370136,25834.038567,63.764394,69.600953
2019-01-01 03:00:00+00:00,296.386236,76.497144,7016.364713,2583.072065,194.776656,155.103687,1093.148093,2241.644174,6127.023839,60.603688,94.495113,819.622205,289.767312,3583.985061,853.443534,3306.702692,26212.497202,26245.554952,63.565901,69.504562
2019-01-01 04:00:00+00:00,297.930061,93.520467,7029.058552,2675.531871,201.577941,158.938178,1098.271321,2338.799631,6138.906619,60.468877,95.065575,991.088364,290.430006,3597.722482,1027.251278,3357.476366,26595.328603,26629.353892,63.354813,69.403318
2019-01-01 05:00:00+00:00,299.394425,109.623295,7037.880163,2762.310591,207.923000,161.220678,1102.900310,2422.224840,6149.837110,60.316880,95.600571,1149.103691,291.052818,3621.113621,1186.100192,3411.481679,26950.437330,26984.796480,63.132633,69.296086
2019-01-01 06:00:00+00:00,300.768688,124.811862,7042.997511,2843.336840,213.826611,162.480823,1107.022570,2493.314663,6159.738927,60.148179,96.097723,1293.230083,291.630977,3652.934736,1329.980128,3468.835851,27277.913129,27311.900712,62.900861,69.182124
2019-01-01 07:00:00+00:00,302.048049,139.106635,7044.612284,2918.668676,219.307389,163.133861,1110.641748,2553.379864,6168.589740,59.963801,96.556126,1423.419669,292.162588,3692.021954,1459.166822,3529.407102,27578.234752,27611.161343,62.660965,69.061005
2019-01-01 08:00:00+00:00,303.231764,152.537661,7042.946049,2988.453884,224.386200,163.501849,1113.772497,2603.639316,6176.404037,59.765117,96.975898,1539.907907,292.647707,3737.281251,1574.139868,3592.899925,27852.169759,27883.413580,62.414359,68.932559
2019-01-01 09:00:00+00:00,304.321838,165.141189,7038.230382,3052.900800,229.085050,163.830945,1116.436736,2645.217027,6183.220687,59.553694,97.357852,1643.132657,293.087685,3787.694123,1675.520671,3658.917047,28100.695801,28129.729873,62.162391,68.796822


In [42]:
final.shape

(17521, 20)

In [43]:
final_df=pd.concat([df,final])

In [44]:
final_df

,gen_biomass,gen_fossil_coal,gen_fossil_gas,gen_fossil_hard_coal,gen_fossil_oil,gen_hydro,gen_hydro_ror,gen_hydro_water,gen_nuclear,gen_other,gen_other_renewable,gen_solar,gen_waste,gen_waste_onshore,forecast_solar_ahead,forecast wind onshore day ahead,total_load_forecast,total_load_actual,price_day_ahead,price_actual
2014-12-31 23:00:00+00:00,447.000000,329.000000,4844.000000,4821.000000,162.000000,863.000000,1051.000000,1899.000000,7096.000000,43.000000,73.000000,49.000000,196.000000,6378.00000,17.000000,6436.000000,26118.00000,25385.00000,50.100000,65.410000
2015-01-01 00:00:00+00:00,449.000000,328.000000,5196.000000,4755.000000,158.000000,920.000000,1009.000000,1658.000000,7096.000000,43.000000,71.000000,50.000000,195.000000,5890.00000,16.000000,5856.000000,24934.00000,24382.00000,48.100000,64.920000
2015-01-01 01:00:00+00:00,448.000000,323.000000,4857.000000,4581.000000,157.000000,1164.000000,973.000000,1371.000000,7099.000000,43.000000,73.000000,50.000000,196.000000,5461.00000,8.000000,5454.000000,23515.00000,22734.00000,47.330000,64.480000
2015-01-01 02:00:00+00:00,438.000000,254.000000,4314.000000,4131.000000,160.000000,1503.000000,949.000000,779.000000,7098.000000,43.000000,75.000000,50.000000,191.000000,5238.00000,2.000000,5151.000000,22642.00000,21286.00000,42.270000,59.320000
2015-01-01 03:00:00+00:00,428.000000,187.000000,4130.000000,3840.000000,156.000000,1826.000000,953.000000,720.000000,7097.000000,43.000000,74.000000,42.000000,189.000000,4935.00000,9.000000,4861.000000,21785.00000,20264.00000,38.410000,56.040000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-30 20:00:00+00:00,382.881538,448.068156,5629.404605,4252.422833,298.162309,474.350347,972.008996,2601.832018,6261.098186,60.136448,85.756346,1433.905953,269.836444,5460.37854,1440.212969,5466.506914,28710.95117,28697.45294,49.915589,57.901342
2020-12-30 21:00:00+00:00,382.881538,448.068156,5629.404605,4252.422833,298.162309,474.350347,972.008996,2601.832018,6261.098186,60.136448,85.756346,1433.905953,269.836444,5460.37854,1440.212969,5466.506914,28710.95117,28697.45294,49.915589,57.901342
2020-12-30 22:00:00+00:00,382.881538,448.068156,5629.404605,4252.422833,298.162309,474.350347,972.008996,2601.832018,6261.098186,60.136448,85.756346,1433.905953,269.836444,5460.37854,1440.212969,5466.506914,28710.95117,28697.45294,49.915589,57.901342
2020-12-30 23:00:00+00:00,382.881538,448.068156,5629.404605,4252.422833,298.162309,474.350347,972.008996,2601.832018,6261.098186,60.136448,85.756346,1433.905953,269.836444,5460.37854,1440.212969,5466.506914,28710.95117,28697.45294,49.915589,57.901342


In [45]:
final_df['gen_biomass']

2014-12-31 23:00:00+00:00    447.000000
2015-01-01 00:00:00+00:00    449.000000
2015-01-01 01:00:00+00:00    448.000000
2015-01-01 02:00:00+00:00    438.000000
2015-01-01 03:00:00+00:00    428.000000
                                ...    
2020-12-30 20:00:00+00:00    382.881538
2020-12-30 21:00:00+00:00    382.881538
2020-12-30 22:00:00+00:00    382.881538
2020-12-30 23:00:00+00:00    382.881538
2020-12-31 00:00:00+00:00    382.881538
Name: gen_biomass, Length: 52585, dtype: float64

In [48]:
prediction_time = '2020-12-30 00:00:00+00:00' 
time = pd.to_datetime(prediction_time, utc=True)
print(time)
print(type(time))
final_df.loc[time]

2020-12-30 00:00:00+00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


gen_biomass                          382.881538
gen_fossil_coal                      448.068156
gen_fossil_gas                      5629.404605
gen_fossil_hard_coal                4252.422833
gen_fossil_oil                       298.162309
gen_hydro                            474.350347
gen_hydro_ror                        972.008996
gen_hydro_water                     2601.832018
gen_nuclear                         6261.098186
gen_other                             60.136448
gen_other_renewable                   85.756346
gen_solar                           1433.905953
gen_waste                            269.836444
gen_waste_onshore                   5460.378540
forecast_solar_ahead                1440.212969
forecast wind onshore day ahead     5466.506914
total_load_forecast                28710.951170
total_load_actual                  28697.452940
price_day_ahead                       49.915589
price_actual                          57.901342
Name: 2020-12-30 00:00:00+00:00, dtype: 

In [ ]:
# #forecasting from 1st Jan 2019
# fc = model_fitted.forecast(y=forecast_input, steps=5000)
# df_forecast = pd.DataFrame(fc, index=df.index[-nobs:], columns=df.columns + '_pred')
# df_forecast.tail()

In [ ]:
# real = df_test['price_actual']
# len(real)

In [ ]:
# predicted = df_forecast['price_actual_pred']
# len(actual)

In [ ]:
# import math
# from sklearn.metrics import mean_squared_error
# math.sqrt( mean_squared_error(predicted,real))

In [ ]:
# import matplotlib.pyplot as plt
# # zoom plot
# plt.figure(figsize=(20,10))
# plt.plot(real, color='red')
# plt.plot(predicted, color='green')
# plt.title('Actual Vs Predicted')
# plt.show()

In [ ]:
# combine = pd.concat([df_test['gen_biomass'], df_forecast['gen_biomass_pred']], axis=1)
# combine['accuracy'] = round(combine.apply(lambda row: row.gen_biomass_pred /
# row.gen_biomass *100, axis = 1),2)
# combine['accuracy'] = pd.Series(["{0:.2f}%".format(val) for val in combine['accuracy']],index = combine.index)
# combine = combine.round(decimals=2)
# combine = combine.reset_index()
# combine

In [ ]:
# combine = pd.concat([df_test['price_actual'], df_forecast['price_actual_pred']], axis=1)
# combine['accuracy'] = round(combine.apply(lambda row: row.price_actual_pred /
# row.price_actual *100, axis = 1),2)
# combine['accuracy'] = pd.Series(["{0:.2f}%".format(val) for val in combine['accuracy']],index = combine.index)
# combine = combine.round(decimals=2)
# combine = combine.reset_index()
# combine.tail()

In [ ]:
# from pandas.tseries.offsets import DateOffset
# ts = pd.Timestamp('2018-12-31 23:00:00+01:00')
# ts + [DateOffset(hours=x) for x in range(0,24)]

In [ ]:
# from pandas.tseries.offsets import DateOffset
# future_dates = [df.index[-1] + DateOffset(hours=x) for x in range(0,24)]

In [ ]:
# pd.DataFrame(ts)

In [ ]:
# new_df = pd.DataFrame(index = ts[1:], columns=df.columns)

In [ ]:
import pickle
filename = 'var_energy.sav'
pickle.dump(model_fitted, open(filename, 'wb'))

In [49]:
final_df.to_csv('final.csv')

In [58]:
time = '2020-08-03 05:00:00+00:00'
time = pd.to_datetime(pred_time, utc=True)
print(time)
final_df.loc[time].to_dict()


{'gen_biomass': 382.8815381520699,
 'gen_fossil_coal': 448.0681555838398,
 'gen_fossil_gas': 5629.404604599546,
 'gen_fossil_hard_coal': 4252.422832735541,
 'gen_fossil_oil': 298.162309290984,
 'gen_hydro': 474.35034670155125,
 'gen_hydro_ror': 972.0089957920554,
 'gen_hydro_water': 2601.8320176443704,
 'gen_nuclear': 6261.098185668016,
 'gen_other': 60.13644810174851,
 'gen_other_renewable': 85.75634576747944,
 'gen_solar': 1433.9059532490412,
 'gen_waste': 269.8364439249283,
 'gen_waste_onshore': 5460.378540368836,
 'forecast_solar_ahead': 1440.2129688545153,
 'forecast wind onshore day ahead': 5466.506914068481,
 'total_load_forecast': 28710.951170463486,
 'total_load_actual': 28697.45293967747,
 'price_day_ahead': 49.91558927505661,
 'price_actual': 57.90134243121983}

In [ ]:
# from statsmodels.tsa.stattools import acf
# def forecast_accuracy(forecast, actual):
#     mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
#     me = np.mean(forecast - actual)             # ME
#     mae = np.mean(np.abs(forecast - actual))    # MAE
#     mpe = np.mean((forecast - actual)/actual)   # MPE
#     rmse = np.mean((forecast - actual)**2)**.5  # RMSE
#     corr = np.corrcoef(forecast, actual)[0,1]   # corr
#     mins = np.amin(np.hstack([forecast[:,None], 
#                               actual[:,None]]), axis=1)
#     maxs = np.amax(np.hstack([forecast[:,None], 
#                               actual[:,None]]), axis=1)
#     minmax = 1 - np.mean(mins/maxs)             # minmax
#     return({'mape':mape, 'me':me, 'mae': mae, 
#             'mpe': mpe, 'rmse':rmse, 'corr':corr, 'minmax':minmax})

In [ ]:
# print('Forecast Accuracy of: gen_biomass')
# accuracy_prod = forecast_accuracy(df_train['generation biomass'].values, df_test['generation biomass'])
# for k, v in accuracy_prod.items():
#     print(adjust(k), ': ', round(v,4))

In [52]:
df


,gen_biomass,gen_fossil_coal,gen_fossil_gas,gen_fossil_hard_coal,gen_fossil_oil,gen_hydro,gen_hydro_ror,gen_hydro_water,gen_nuclear,gen_other,gen_other_renewable,gen_solar,gen_waste,gen_waste_onshore,forecast_solar_ahead,forecast wind onshore day ahead,total_load_forecast,total_load_actual,price_day_ahead,price_actual
time,,,,,,,,,,,,,,,,,,,,
2014-12-31 23:00:00+00:00,447.0,329.0,4844.0,4821.0,162.0,863.0,1051.0,1899.0,7096.0,43.0,73.0,49.0,196.0,6378.0,17.0,6436.0,26118.0,25385.0,50.10,65.41
2015-01-01 00:00:00+00:00,449.0,328.0,5196.0,4755.0,158.0,920.0,1009.0,1658.0,7096.0,43.0,71.0,50.0,195.0,5890.0,16.0,5856.0,24934.0,24382.0,48.10,64.92
2015-01-01 01:00:00+00:00,448.0,323.0,4857.0,4581.0,157.0,1164.0,973.0,1371.0,7099.0,43.0,73.0,50.0,196.0,5461.0,8.0,5454.0,23515.0,22734.0,47.33,64.48
2015-01-01 02:00:00+00:00,438.0,254.0,4314.0,4131.0,160.0,1503.0,949.0,779.0,7098.0,43.0,75.0,50.0,191.0,5238.0,2.0,5151.0,22642.0,21286.0,42.27,59.32
2015-01-01 03:00:00+00:00,428.0,187.0,4130.0,3840.0,156.0,1826.0,953.0,720.0,7097.0,43.0,74.0,42.0,189.0,4935.0,9.0,4861.0,21785.0,20264.0,38.41,56.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 18:00:00+00:00,297.0,0.0,7634.0,2628.0,178.0,1.0,1135.0,4836.0,6073.0,63.0,95.0,85.0,277.0,3113.0,96.0,3253.0,30619.0,30653.0,68.85,77.02
2018-12-31 19:00:00+00:00,296.0,0.0,7241.0,2566.0,174.0,1.0,1172.0,3931.0,6074.0,62.0,95.0,33.0,280.0,3288.0,51.0,3353.0,29932.0,29735.0,68.40,76.16
2018-12-31 20:00:00+00:00,292.0,0.0,7025.0,2422.0,168.0,50.0,1148.0,2831.0,6076.0,61.0,94.0,31.0,286.0,3503.0,36.0,3404.0,27903.0,28071.0,66.88,74.30


In [53]:
final_df

,gen_biomass,gen_fossil_coal,gen_fossil_gas,gen_fossil_hard_coal,gen_fossil_oil,gen_hydro,gen_hydro_ror,gen_hydro_water,gen_nuclear,gen_other,gen_other_renewable,gen_solar,gen_waste,gen_waste_onshore,forecast_solar_ahead,forecast wind onshore day ahead,total_load_forecast,total_load_actual,price_day_ahead,price_actual
2014-12-31 23:00:00+00:00,447.000000,329.000000,4844.000000,4821.000000,162.000000,863.000000,1051.000000,1899.000000,7096.000000,43.000000,73.000000,49.000000,196.000000,6378.00000,17.000000,6436.000000,26118.00000,25385.00000,50.100000,65.410000
2015-01-01 00:00:00+00:00,449.000000,328.000000,5196.000000,4755.000000,158.000000,920.000000,1009.000000,1658.000000,7096.000000,43.000000,71.000000,50.000000,195.000000,5890.00000,16.000000,5856.000000,24934.00000,24382.00000,48.100000,64.920000
2015-01-01 01:00:00+00:00,448.000000,323.000000,4857.000000,4581.000000,157.000000,1164.000000,973.000000,1371.000000,7099.000000,43.000000,73.000000,50.000000,196.000000,5461.00000,8.000000,5454.000000,23515.00000,22734.00000,47.330000,64.480000
2015-01-01 02:00:00+00:00,438.000000,254.000000,4314.000000,4131.000000,160.000000,1503.000000,949.000000,779.000000,7098.000000,43.000000,75.000000,50.000000,191.000000,5238.00000,2.000000,5151.000000,22642.00000,21286.00000,42.270000,59.320000
2015-01-01 03:00:00+00:00,428.000000,187.000000,4130.000000,3840.000000,156.000000,1826.000000,953.000000,720.000000,7097.000000,43.000000,74.000000,42.000000,189.000000,4935.00000,9.000000,4861.000000,21785.00000,20264.00000,38.410000,56.040000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-30 20:00:00+00:00,382.881538,448.068156,5629.404605,4252.422833,298.162309,474.350347,972.008996,2601.832018,6261.098186,60.136448,85.756346,1433.905953,269.836444,5460.37854,1440.212969,5466.506914,28710.95117,28697.45294,49.915589,57.901342
2020-12-30 21:00:00+00:00,382.881538,448.068156,5629.404605,4252.422833,298.162309,474.350347,972.008996,2601.832018,6261.098186,60.136448,85.756346,1433.905953,269.836444,5460.37854,1440.212969,5466.506914,28710.95117,28697.45294,49.915589,57.901342
2020-12-30 22:00:00+00:00,382.881538,448.068156,5629.404605,4252.422833,298.162309,474.350347,972.008996,2601.832018,6261.098186,60.136448,85.756346,1433.905953,269.836444,5460.37854,1440.212969,5466.506914,28710.95117,28697.45294,49.915589,57.901342
2020-12-30 23:00:00+00:00,382.881538,448.068156,5629.404605,4252.422833,298.162309,474.350347,972.008996,2601.832018,6261.098186,60.136448,85.756346,1433.905953,269.836444,5460.37854,1440.212969,5466.506914,28710.95117,28697.45294,49.915589,57.901342


In [68]:
mydf = pd.read_csv('final.csv', index_col='Unnamed: 0')
mydf

,gen_biomass,gen_fossil_coal,gen_fossil_gas,gen_fossil_hard_coal,gen_fossil_oil,gen_hydro,gen_hydro_ror,gen_hydro_water,gen_nuclear,gen_other,gen_other_renewable,gen_solar,gen_waste,gen_waste_onshore,forecast_solar_ahead,forecast wind onshore day ahead,total_load_forecast,total_load_actual,price_day_ahead,price_actual
2014-12-31 23:00:00+00:00,447.000000,329.000000,4844.000000,4821.000000,162.000000,863.000000,1051.000000,1899.000000,7096.000000,43.000000,73.000000,49.000000,196.000000,6378.00000,17.000000,6436.000000,26118.00000,25385.00000,50.100000,65.410000
2015-01-01 00:00:00+00:00,449.000000,328.000000,5196.000000,4755.000000,158.000000,920.000000,1009.000000,1658.000000,7096.000000,43.000000,71.000000,50.000000,195.000000,5890.00000,16.000000,5856.000000,24934.00000,24382.00000,48.100000,64.920000
2015-01-01 01:00:00+00:00,448.000000,323.000000,4857.000000,4581.000000,157.000000,1164.000000,973.000000,1371.000000,7099.000000,43.000000,73.000000,50.000000,196.000000,5461.00000,8.000000,5454.000000,23515.00000,22734.00000,47.330000,64.480000
2015-01-01 02:00:00+00:00,438.000000,254.000000,4314.000000,4131.000000,160.000000,1503.000000,949.000000,779.000000,7098.000000,43.000000,75.000000,50.000000,191.000000,5238.00000,2.000000,5151.000000,22642.00000,21286.00000,42.270000,59.320000
2015-01-01 03:00:00+00:00,428.000000,187.000000,4130.000000,3840.000000,156.000000,1826.000000,953.000000,720.000000,7097.000000,43.000000,74.000000,42.000000,189.000000,4935.00000,9.000000,4861.000000,21785.00000,20264.00000,38.410000,56.040000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-30 20:00:00+00:00,382.881538,448.068156,5629.404605,4252.422833,298.162309,474.350347,972.008996,2601.832018,6261.098186,60.136448,85.756346,1433.905953,269.836444,5460.37854,1440.212969,5466.506914,28710.95117,28697.45294,49.915589,57.901342
2020-12-30 21:00:00+00:00,382.881538,448.068156,5629.404605,4252.422833,298.162309,474.350347,972.008996,2601.832018,6261.098186,60.136448,85.756346,1433.905953,269.836444,5460.37854,1440.212969,5466.506914,28710.95117,28697.45294,49.915589,57.901342
2020-12-30 22:00:00+00:00,382.881538,448.068156,5629.404605,4252.422833,298.162309,474.350347,972.008996,2601.832018,6261.098186,60.136448,85.756346,1433.905953,269.836444,5460.37854,1440.212969,5466.506914,28710.95117,28697.45294,49.915589,57.901342
2020-12-30 23:00:00+00:00,382.881538,448.068156,5629.404605,4252.422833,298.162309,474.350347,972.008996,2601.832018,6261.098186,60.136448,85.756346,1433.905953,269.836444,5460.37854,1440.212969,5466.506914,28710.95117,28697.45294,49.915589,57.901342


In [69]:
parsed_date = '2020-08-03 05:00:00+00:00'
mydf.loc[parsed_date]

gen_biomass                          382.881538
gen_fossil_coal                      448.068156
gen_fossil_gas                      5629.404605
gen_fossil_hard_coal                4252.422833
gen_fossil_oil                       298.162309
gen_hydro                            474.350347
gen_hydro_ror                        972.008996
gen_hydro_water                     2601.832018
gen_nuclear                         6261.098186
gen_other                             60.136448
gen_other_renewable                   85.756346
gen_solar                           1433.905953
gen_waste                            269.836444
gen_waste_onshore                   5460.378540
forecast_solar_ahead                1440.212969
forecast wind onshore day ahead     5466.506914
total_load_forecast                28710.951170
total_load_actual                  28697.452940
price_day_ahead                       49.915589
price_actual                          57.901342
Name: 2020-08-03 05:00:00+00:00, dtype: 